In [ ]:
### Задание 4.1: Кастомный итератор

class FibonacciIterator:
    def __init__(self, max_iterations):
        self.max_iterations = max_iterations
        self.count = 0
        self.previous = 0
        self.current = 1

    def __iter__(self):
        return self

    def __next__(self):
        if self.count >= self.max_iterations:
            raise StopIteration

        result = self.previous
        self.previous, self.current = self.current, self.previous + self.current
        self.count += 1
        return result

In [ ]:
### Задание 4.2: Генератор простых чисел

limit = int(input('Введите значение, сколько нужно сгенерировать чисел:\n'))

def find_primes(limit):
    count = 0
    n = 2
    while True:
        for i in range(2, int(n ** 0.5) + 1):
            if n % i == 0:
                break
        else:
            yield n
            count += 1
            if count >= limit:
                return
        n += 1

for prime in find_primes(limit):
    print(prime)

In [ ]:
### Задание 4.3: Генератор с send()

def squares_generator():

    current = 0
    iteration = 0
    
    print(f"[LOG] Генератор инициализирован. Начальное значение: {current}")
    
    while True:
        # возвращаем квадрат текущего числа
        result = current ** 2
        print(f"[LOG] Итерация {iteration}: {current}² = {result}")
        
        # получаем новое значение через send() или продолжаем последовательность
        received = yield result
        
        if received is not None:
            # если получено новое значение через send() - сбрасываем генератор
            old_value = current
            current = received
            iteration = 0
            print(f"[LOG] Получено новое значение через send(): {received}")
            print(f"[LOG] Генератор сброшен: {old_value} -> {current}")
        else:
            # иначе увеличиваем текущее значение для следующей итерации
            current += 1
        
        iteration += 1

In [ ]:
### Задание 4.4: Рекурсивный генератор

class treenode:
    def __init__(self, value):
        self.value = value
        self.children = []
    
    def add_child(self, child_node):
        self.children.append(child_node)
    
    def traverse_depth(self):
        yield self.value
        for child in self.children:
            yield from child.traverse_depth()
    
    def traverse_breadth(self):
        from collections import deque
        queue = deque([self])
        while queue:
            node = queue.popleft()
            yield node.value
            queue.extend(node.children)

In [ ]:
### Задание 5.1: Система логирования для веб-приложения

from collections import namedtuple, deque, defaultdict
import json
import time
import os

# структура лога для веб-приложения
WebLogEntry = namedtuple('WebLogEntry', [
    'timestamp', 'level', 'message', 'component', 'operation', 
    'status', 'duration', 'user_id', 'ip_address'
])

# система логирования для веб-приложения
class WebLogger:
    def __init__(self, component_name, max_logs=1000, rotation_interval=3600):
        self.component_name = component_name
        self.history_logs = deque(maxlen=max_logs)
        self.grouped_logs = defaultdict(list)
        self.rotation_interval = rotation_interval
        self.last_rotation = time.time()
    
    def _check_rotation(self):
        """проверяет необходимость ротации логов по времени"""
        current_time = time.time()
        if current_time - self.last_rotation >= self.rotation_interval:
            self._rotate_logs()
            self.last_rotation = current_time
    
    def _rotate_logs(self):
        """выполняет ротацию логов - сохраняет старые логи и очищает текущие"""
        if self.history_logs:
            filename = f"logs/{self.component_name}_{int(time.time())}.json"
            os.makedirs('logs', exist_ok=True)
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump([log._asdict() for log in self.history_logs], f, ensure_ascii=False, indent=2)
            
            self.history_logs.clear()
            self.grouped_logs.clear()
    
    def create_log_entry(self, level, message, operation, status, duration=0, user_id=None, ip_address=None):
        """создает запись лога для веб-приложения"""
        return WebLogEntry(
            timestamp=time.time(),
            level=level,
            message=message,
            component=self.component_name,
            operation=operation,
            status=status,
            duration=duration,
            user_id=user_id,
            ip_address=ip_address
        )
    
    def log(self, level, message, operation, status, duration=0, user_id=None, ip_address=None):
        """добавляет лог в систему с проверкой ротации"""
        self._check_rotation()
        
        log_entry = self.create_log_entry(level, message, operation, status, duration, user_id, ip_address)
        
        self.history_logs.appendleft(log_entry)
        self.grouped_logs[log_entry.level].append(log_entry)
        self._print_json_log(log_entry)
        
        return log_entry
    
    def _print_json_log(self, log_entry):
        """выводит лог в json формате"""
        from datetime import datetime
        log_dict = {
            'timestamp': datetime.fromtimestamp(log_entry.timestamp).isoformat(),
            'level': log_entry.level,
            'message': log_entry.message,
            'component': log_entry.component,
            'operation': log_entry.operation,
            'status': log_entry.status,
            'duration_ms': log_entry.duration,
            'user_id': log_entry.user_id,
            'ip_address': log_entry.ip_address
        }
        print(json.dumps(log_dict, ensure_ascii=False))

# менеджер логгеров для разных компонентов
class LoggerManager:
    def __init__(self):
        self.loggers = {}
    
    def get_logger(self, component_name):
        """возвращает логгер для указанного компонента"""
        if component_name not in self.loggers:
            self.loggers[component_name] = WebLogger(component_name)
        return self.loggers[component_name]

# декоратор для логирования операций api
def log_api_request(logger, operation_name):
    def decorator(func):
        def wrapper(*args, **kwargs):
            start_time = time.time()
            user_id = kwargs.get('user_id')
            ip_address = kwargs.get('ip_address')
            
            logger.log(
                level="INFO",
                message=f"starting {operation_name}",
                operation=operation_name,
                status="started",
                user_id=user_id,
                ip_address=ip_address
            )
            
            try:
                result = func(*args, **kwargs)
                duration = (time.time() - start_time) * 1000
                
                logger.log(
                    level="INFO", 
                    message=f"completed {operation_name} successfully",
                    operation=operation_name,
                    status="success",
                    duration=duration,
                    user_id=user_id,
                    ip_address=ip_address
                )
                return result
                
            except Exception as e:
                duration = (time.time() - start_time) * 1000
                
                logger.log(
                    level="ERROR",
                    message=f"error in {operation_name}: {str(e)}",
                    operation=operation_name,
                    status="error",
                    duration=duration,
                    user_id=user_id,
                    ip_address=ip_address
                )
                raise
        return wrapper
    return decorator

In [ ]:
### Задание 5.2: Логирование с контекстом

from collections import namedtuple, deque, defaultdict, Counter, OrderedDict
import json
import time
import os

# структура лога с контекстной информацией
LogEntry = namedtuple('LogEntry', [
    'timestamp', 'level', 'message', 'module', 'function', 
    'user_id', 'request_id'
])

# контекст для хранения информации о запросе
class RequestContext:
    def __init__(self, user_id=None):
        self.user_id = user_id
        self.request_id = f"req_{int(time.time())}_{os.urandom(4).hex()}"

# генератор уникальных идентификаторов запросов
class SimpleIDGenerator:
    def __init__(self):
        self.counter = 0
    
    def generate_id(self):
        self.counter += 1
        return f"req_{self.counter}_{int(time.time())}"

# система логирования с контекстом
class Logger:
    def __init__(self):
        self.history_logs = deque(maxlen=100)
        self.grouped_logs = defaultdict(list)
        self.id_generator = SimpleIDGenerator()
    
    def create_log_entry(self, level, message, module, function, context):
        """создает запись лога с контекстом"""
        return LogEntry(
            timestamp=time.time(),
            level=level,
            message=message,
            module=module,
            function=function,
            user_id=context.user_id,
            request_id=context.request_id
        )
    
    def log(self, level, message, module, function, context):
        """добавляет лог в систему и выводит в json формате"""
        log_entry = self.create_log_entry(level, message, module, function, context)
        
        # сохраняем в историю
        self.history_logs.appendleft(log_entry)
        
        # группируем по уровням
        self.grouped_logs[log_entry.level].append(log_entry)
        
        # выводим в json формате
        self._print_json_log(log_entry)
        
        return log_entry
    
    def _print_json_log(self, log_entry):
        """выводит лог в json формате с читаемым timestamp"""
        from datetime import datetime
        log_dict = {
            'timestamp': datetime.fromtimestamp(log_entry.timestamp).isoformat(),
            'level': log_entry.level,
            'message': log_entry.message,
            'module': log_entry.module,
            'function': log_entry.function,
            'user_id': log_entry.user_id,
            'request_id': log_entry.request_id
        }
        print(json.dumps(log_dict, ensure_ascii=False, indent=2))

# декоратор для автоматического логирования вызовов функций
def log_function(logger, context):
    def decorator(func):
        def wrapper(*args, **kwargs):
            logger.log(
                level="INFO",
                message=f"starting {func.__name__}",
                module=func.__module__ if hasattr(func, '__module__') else '__main__',
                function=func.__name__,
                context=context
            )
            
            try:
                result = func(*args, **kwargs)
                logger.log(
                    level="INFO", 
                    message=f"completed {func.__name__} successfully",
                    module=func.__module__ if hasattr(func, '__module__') else '__main__',
                    function=func.__name__,
                    context=context
                )
                return result
            except Exception as e:
                logger.log(
                    level="ERROR",
                    message=f"error in {func.__name__}: {str(e)}",
                    module=func.__module__ if hasattr(func, '__module__') else '__main__',
                    function=func.__name__,
                    context=context
                )
                raise
        return wrapper
    return decorator

In [ ]:
### Задание 5.3: Мониторинг производительности

import sys
import os
import time
import json
import pickle
from collections import namedtuple, deque, defaultdict, Counter, OrderedDict
from datetime import datetime

# NamedTuple для метрик
PerformanceMetric = namedtuple("PerformanceMetric", ["function_name", "execution_time", "memory_usage", "timestamp"])

# очередь для последних 100 метрик
recent_metrics = deque(maxlen=100)

# группировка по функциям
function_metrics = defaultdict(list)

# счетчик вызовов функций
function_calls = Counter()

# хронологическое хранилище
chronological_metrics = OrderedDict()

# запись метрики
def record_metric(function_name, start_time, result):
    end_time = time.time()
    execution_time = end_time - start_time
    memory_usage = sys.getsizeof(result)
    timestamp = datetime.now().isoformat()

    metric = PerformanceMetric(function_name, execution_time, memory_usage, timestamp)

    recent_metrics.append(metric)
    function_metrics[function_name].append(metric)
    function_calls[function_name] += 1
    chronological_metrics[timestamp] = metric

# получение статистики по функции
def get_function_stats(function_name):
    metrics = function_metrics.get(function_name, [])
    if not metrics:
        return f"No metrics recorded for function '{function_name}'"

    total_exec_time = sum(m.execution_time for m in metrics)
    total_memory = sum(m.memory_usage for m in metrics)
    count = len(metrics)

    return {
        "function_name": function_name,
        "calls": function_calls[function_name],
        "avg_execution_time": total_exec_time / count,
        "avg_memory_usage": total_memory / count
    }

# получение использования памяти
def get_memory_usage():
    return sum(sys.getsizeof(m) for m in recent_metrics)

# экспорт метрик
def export_metrics(directory="metrics_output"):
    os.makedirs(directory, exist_ok=True)

    # JSON
    json_path = os.path.join(directory, "metrics.json")
    with open(json_path, "w") as f:
        json.dump([m._asdict() for m in recent_metrics], f, indent=4)

    # Pickle
    pickle_path = os.path.join(directory, "metrics.pkl")
    with open(pickle_path, "wb") as f:
        pickle.dump(list(recent_metrics), f)

    print(f"Metrics exported to {directory}/ (JSON and Pickle formats)")


In [ ]:
### Задание 6.1: Чистые функции
#Создайте набор чистых функций для работы с математическими операциями:
#- Функции для базовых операций (сложение, вычитание, умножение, деление)
#- Функции для работы с векторами (скалярное произведение, норма)
#- Функции для работы с матрицами (транспонирование, умножение)
#- Убедитесь, что все функции являются чистыми (детерминированными и без побочных эффектов)

In [ ]:
### Задание 6.2: Lambda функции и высшие функции
#Реализуйте следующие задачи используя lambda функции:
#- Создайте список функций-преобразований (квадрат, куб, факториал)
#- Примените эти функции к списку чисел используя map()
#- Отфильтруйте числа по различным условиям используя filter()
#- Вычислите сумму, произведение, максимум, минимум используя reduce()

In [ ]:
### Задание 6.3: Функциональные композиции
#Создайте систему для композиции функций:
#- Реализуйте функцию compose(), которая принимает несколько функций и возвращает их композицию
#- Создайте pipeline для обработки данных (очистка, трансформация, валидация)
#- Используйте partial() для создания специализированных версий функций
#- Примените currying для создания функций с частичным применением аргументов